In [1]:
! pip install -U accelerate
! pip install -U transformers

In [2]:
!pip install evaluate

In [3]:
import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

import warnings
warnings.filterwarnings("ignore")

In [4]:
DEVICE = "cuda:0"

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Salesforce/dialogstudio-t5-base-v1.0")
model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/dialogstudio-t5-base-v1.0")

input_text = "Answer the following yes/no question by reasoning step-by-step. Can you write 200 words in a single tweet?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

It is not possible to write 200 words in a tweet. Therefore, the final answer is no.


In [6]:
OPTIMIZER = Adam(model.parameters(), lr=0.00001)
Q_LEN = 256   # Question Length
T_LEN = 32    # Target Length
BATCH_SIZE = 4

In [7]:
from datasets import load_dataset
dataset = load_dataset("amaydle/npc-dialogue")

In [8]:
def prepare_data(data):
    articles = []
    for i in range(len(dataset['train']['Query'])):
      context = dataset['train']["Name"][i] + ":" + dataset['train']["Biography"][i]
      question = dataset['train']["Query"][i]
      answer = dataset['train']["Response"][i]
      #context = dataset['context']
      #question = dataset['question']
      #answers = dataset['answers']
      inputs = {"context": context, "question": question, "answer": answer}

      articles.append(inputs)

    return articles

In [9]:
data = prepare_data(dataset)

In [10]:
data = pd.DataFrame(data)

In [11]:
data.head(1)

,context,question,answer
0,Bikram:Bikram is a rough and tough smuggler fr...,What is your opinion on friendship?,Friendship is a bond stronger than blood.


In [12]:
class QA_Dataset(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe
        self.questions = self.data["question"]
        self.context = self.data["context"]
        self.answer = self.data['answer']

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.context[idx]
        answer = self.answer[idx]

        question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                                    truncation=True, pad_to_max_length=True, add_special_tokens=True)
        answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length",
                                          truncation=True, pad_to_max_length=True, add_special_tokens=True)

        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100

        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
        }

In [13]:
# Dataloader

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

#train_sampler = RandomSampler(train_data.index)
#val_sampler = RandomSampler(val_data.index)

train_sampler = train_data.index
val_sampler = val_data.index

qa_dataset = QA_Dataset(tokenizer, data, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

In [14]:
train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0

for epoch in range(500):
    model.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)
        model.to('cuda')
        outputs = model(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        train_loss += outputs.loss.item()
        train_batch_count += 1

    #Evaluation
    model.eval()
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = model(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        val_loss += outputs.loss.item()
        val_batch_count += 1

    print(f"\n{epoch+1}/{2} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss/val_batch_count}")

Validation batches: 100%|█████████████████████████████████████████████████████████████████████████| 87/87 [00:12<00:00,  7.22it/s]



1/2 -> Train loss: 2.7310888553011243	Validation loss: 2.3547704315733635


Validation batches: 100%|█████████████████████████████████████████████████████████████████████████| 87/87 [00:12<00:00,  7.23it/s]



2/2 -> Train loss: 2.6128623988317408	Validation loss: 2.2500726217511056


Validation batches: 100%|█████████████████████████████████████████████████████████████████████████| 87/87 [00:12<00:00,  7.21it/s]


...


498/2 -> Train loss: 0.19865781877507213	Validation loss: 0.058205067715794034


Validation batches: 100%|█████████████████████████████████████████████████████████████████████████| 87/87 [00:12<00:00,  7.21it/s]



499/2 -> Train loss: 0.19827028746079328	Validation loss: 0.05808909019631567


Validation batches: 100%|█████████████████████████████████████████████████████████████████████████| 87/87 [00:12<00:00,  7.23it/s]


500/2 -> Train loss: 0.197882811320639	Validation loss: 0.057973286566180664


In [15]:
def predict_answer(context, question, ref_answer=None, metric=None):
    inputs = tokenizer(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)

    predicted_answer = tokenizer.decode(outputs.flatten(), skip_special_tokens=True)

    if ref_answer:
      if metric == "bleu":
        # Load the Bleu metric
        bleu = evaluate.load("google_bleu")
        score = bleu.compute(predictions=[predicted_answer],
                            references=[ref_answer])

        print("\nContext: \n", context)
        #print("\n")
        print("Question: \n", question)
        return {
            "Reference Answer: ": ref_answer,
            "Predicted Answer: ": predicted_answer,
            "BLEU Score: ": score
        }
      elif metric == "rouge":
        rouge = evaluate.load("rouge")
        score = rouge.compute(predictions=[predicted_answer],
                            references=[ref_answer])

        print("\nContext: \n", context)
        #print("\n")
        print("Question: \n", question)
        return {
            "Reference Answer: ": ref_answer,
            "Predicted Answer: ": predicted_answer,
            "BLEU Score: ": score
        }
    else:
        return predicted_answer

In [16]:
ref_answers = []
predictions = []
for i in range(len(dataset['test']['Biography'])):
  context = dataset['test']["Name"][i] + ":" + dataset['test']['Biography'][i]
  question = dataset['test']['Query'][i]
  ref_answers.append(dataset['test']['Response'][i])

  #print("\n--context--", context)
  #print(predict_answer(context, question, ref_answer=ref_answer))
  predictions.append(predict_answer(context, question))

In [17]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=a96d640b8159dab6a3b0a49fd65fc7beadbadadd33c4e7a1a02fcfc1bf67cc10
  Stored in directory: /home/ubuntu/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [18]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=predictions, references=ref_answers)
print(results)

{'rouge1': 0.275134202609655, 'rouge2': 0.11065895909797585, 'rougeL': 0.24712275364973701, 'rougeLsum': 0.24791197282094923}


In [20]:
model.save_pretrained("/home/ubuntu/ML/datasci-210-capstone-fall23/DialogStudio/dialogstudio_torch_model", from_pt=True)

In [22]:
from transformers import T5ForConditionalGeneration

model_test = T5ForConditionalGeneration.from_pretrained("/home/ubuntu/ML/datasci-210-capstone-fall23/DialogStudio/dialogstudio_torch_model")

In [24]:
!pip install torchsummary

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
from torchsummary import summary

In [30]:
!pip install torchinfo

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for torchinfo from https://files.pythonhosted.org/packages/72/25/973bd6128381951b23cdcd8a9870c6dcfc5606cb864df8eabd82e529f9c1/torchinfo-1.8.0-py3-none-any.whl.metadata


In [33]:
total_params = sum(
	param.numel() for param in model_test.parameters()
)


In [34]:
total_params

247539456

In [38]:
!pip install prettytable

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for prettytable from https://files.pythonhosted.org/packages/4d/81/316b6a55a0d1f327d04cc7b0ba9d04058cb62de6c3a4d4b0df280cbe3b0b/prettytable-3.9.0-py3-none-any.whl.metadata


In [39]:
from prettytable import PrettyTable
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(model_test)

+----------------------------------------------------------------------+------------+
|                               Modules                                | Parameters |
+----------------------------------------------------------------------+------------+
|                            shared.weight                             |  24655104  |
|            encoder.block.0.layer.0.SelfAttention.q.weight            |   589824   |
|            encoder.block.0.layer.0.SelfAttention.k.weight            |   589824   |
|            encoder.block.0.layer.0.SelfAttention.v.weight            |   589824   |
|            encoder.block.0.layer.0.SelfAttention.o.weight            |   589824   |
| encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight |    384     |
|              encoder.block.0.layer.0.layer_norm.weight               |    768     |
|          encoder.block.0.layer.1.DenseReluDense.wi_0.weight          |  1572864   |
|          encoder.block.0.layer.1.DenseReluDense.wi_1

247539456